# imports

In [1]:
import os
import numpy as np
import nibabel as nib
from nilearn import input_data, datasets
import logging
from scipy.stats import skew, kurtosis
from skimage.feature import graycomatrix, graycoprops
from sklearn.preprocessing import StandardScaler
import warnings
import pandas as pd

# Setup logging

In [2]:

logging.basicConfig(filename='processing_log.log', level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')

# Define data paths

In [3]:

mri_skullstrip_data_path = r"C:\Users\ishsh\OneDrive\Desktop\QML_Project\data\processed\mri\derivatives\cleaned_skullstrip"
pet_processed_data_path = r"C:\Users\ishsh\OneDrive\Desktop\QML_Project\data\processed\mri_pet\derivatives\preprocessed_pet"
mri_pet_anat_data_path = r"C:\Users\ishsh\OneDrive\Desktop\QML_Project\data\processed\mri_pet\derivatives\cleaned_skullstrip"
atlas_filename = datasets.fetch_atlas_aal().maps

[get_dataset_dir] Dataset found in C:\Users\ishsh\nilearn_data\aal_SPM12

# Extract ROI

In [4]:
def extract_roi_features(image_path, atlas_filename, surviving_labels=None):
    try:
        img = nib.load(image_path)
        masker = input_data.NiftiLabelsMasker(labels_img=atlas_filename, standardize=False)
        roi_data = masker.fit_transform(img)

        all_roi_features = []
        if surviving_labels is None:
            surviving_labels = range(roi_data.shape[1])

        for i, roi_values in enumerate(roi_data.T):
            if i in surviving_labels:
                mean_val = np.mean(roi_values)
                std_val = np.std(roi_values)
                skew_val = skew(roi_values)
                kurt_val = kurtosis(roi_values)

                if roi_values.max() == roi_values.min():
                    roi_features = [mean_val, std_val, skew_val, kurt_val, 0, 0, 0, 0]
                else:
                    roi_values_int = (roi_values - roi_values.min()) / (roi_values.max() - roi_values.min()) * 255
                    roi_values_int = roi_values_int.astype(np.uint8)

                    size = int(np.sqrt(len(roi_values_int)))
                    if size * size != len(roi_values_int):
                        size += 1
                    reshaped_roi_values = np.pad(roi_values_int, (0, size * size - len(roi_values_int)), 'constant').reshape(size, size)

                    glcm = graycomatrix(reshaped_roi_values, distances=[1], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4], levels=256, symmetric=True, normed=True)
                    contrast = graycoprops(glcm, 'contrast').mean()
                    correlation = graycoprops(glcm, 'correlation').mean()
                    energy = graycoprops(glcm, 'energy').mean()
                    homogeneity = graycoprops(glcm, 'homogeneity').mean()

                    roi_features = [mean_val, std_val, skew_val, kurt_val, contrast, correlation, energy, homogeneity]
                all_roi_features.extend(roi_features)

        return np.array(all_roi_features), surviving_labels

    except FileNotFoundError:
        logging.error(f"Error: File not found at {image_path}")
        return None, None
    except Exception as e:
        logging.error(f"Error processing image {image_path}: {e}")
        return None, None

# Process MRI (Skull-stripped) data

In [5]:

mri_only_data = []
reference_mri_skullstrip_path = None
max_rois_mri_skullstrip = 0

for subject_folder in os.listdir(mri_skullstrip_data_path):
    if subject_folder.startswith("sub-"):
        mri_path = os.path.join(mri_skullstrip_data_path, subject_folder, "anat", f"{subject_folder}_T1w_skullstripped.nii.gz")
        try:
            img = nib.load(mri_path)
            masker = input_data.NiftiLabelsMasker(labels_img=atlas_filename, standardize=False)
            roi_data = masker.fit_transform(img)
            num_rois = roi_data.shape[1]

            if num_rois > max_rois_mri_skullstrip:
                max_rois_mri_skullstrip = num_rois
                reference_mri_skullstrip_path = mri_path
            print(f"(Reference MRI-Only) Subject: {subject_folder}, MRI Path: {mri_path}, ROI Data Shape: {roi_data.shape}") # DEBUG
        except FileNotFoundError:
            logging.warning(f"MRI file not found for subject: {subject_folder} in {mri_path}")
        except Exception as e:
            logging.error(f"Error loading or processing MRI image {mri_path}: {e}")

if reference_mri_skullstrip_path:
    _, surviving_mri_skullstrip_labels = extract_roi_features(reference_mri_skullstrip_path, atlas_filename)
    print(f"Surviving MRI-Only labels (after reference): {surviving_mri_skullstrip_labels}") # DEBUG

    for subject_folder in os.listdir(mri_skullstrip_data_path):
        if subject_folder.startswith("sub-"):
            mri_path = os.path.join(mri_skullstrip_data_path, subject_folder, "anat", f"{subject_folder}_T1w_skullstripped.nii.gz")
            features, _ = extract_roi_features(mri_path, atlas_filename, surviving_mri_skullstrip_labels)
            if features is not None:
                subject_id = subject_folder
                try:
                    numeric_id = int(subject_id.replace("sub-", ""))
                    label = 1 if numeric_id < 4000 else 0
                except ValueError:
                    logging.warning(f"Could not extract numeric ID from {subject_folder}. Assigning default label 0.")
                    label = 0
                feature_dict = {f'M{i+1}': val for i, val in enumerate(features.flatten())}
                mri_only_data.append({'subject_id': subject_id, **feature_dict, 'label': label})
else:
    logging.error("No reference MRI found for MRI-only data. Skipping MRI-only processing.")

print(f"MRI-PET (Anat) Data Path: {mri_pet_anat_data_path}") # DEBUG
print(f"PET (Processed) Data Path: {pet_processed_data_path}") # DEBUG

(Reference MRI-Only) Subject: sub-1, MRI Path: C:\Users\ishsh\OneDrive\Desktop\QML_Project\data\processed\mri\derivatives\cleaned_skullstrip\sub-1\anat\sub-1_T1w_skullstripped.nii.gz, ROI Data Shape: (1, 116)


c:\Users\ishsh\OneDrive\Desktop\QML_Project\venv4qml\Lib\site-packages\nilearn\maskers\nifti_labels_masker.py:888: UserWarning: After resampling the label image to the data image, the following labels were removed: {5102}. Label image only contains 116 labels (including background).
  warnings.warn(


(Reference MRI-Only) Subject: sub-10, MRI Path: C:\Users\ishsh\OneDrive\Desktop\QML_Project\data\processed\mri\derivatives\cleaned_skullstrip\sub-10\anat\sub-10_T1w_skullstripped.nii.gz, ROI Data Shape: (1, 115)


c:\Users\ishsh\OneDrive\Desktop\QML_Project\venv4qml\Lib\site-packages\nilearn\maskers\nifti_labels_masker.py:888: UserWarning: After resampling the label image to the data image, the following labels were removed: {5101, 5102}. Label image only contains 115 labels (including background).
  warnings.warn(


(Reference MRI-Only) Subject: sub-100, MRI Path: C:\Users\ishsh\OneDrive\Desktop\QML_Project\data\processed\mri\derivatives\cleaned_skullstrip\sub-100\anat\sub-100_T1w_skullstripped.nii.gz, ROI Data Shape: (1, 114)


...

(Reference MRI-Only) Subject: sub-92, MRI Path: C:\Users\ishsh\OneDrive\Desktop\QML_Project\data\processed\mri\derivatives\cleaned_skullstrip\sub-92\anat\sub-92_T1w_skullstripped.nii.gz, ROI Data Shape: (1, 110)
(Reference MRI-Only) Subject: sub-93, MRI Path: C:\Users\ishsh\OneDrive\Desktop\QML_Project\data\processed\mri\derivatives\cleaned_skullstrip\sub-93\anat\sub-93_T1w_skullstripped.nii.gz, ROI Data Shape: (1, 116)
(Reference MRI-Only) Subject: sub-94, MRI Path: C:\Users\ishsh\OneDrive\Desktop\QML_Project\data\processed\mri\derivatives\cleaned_skullstrip\sub-94\anat\sub-94_T1w_skullstripped.nii.gz, ROI Data Shape: (1, 116)
(Reference MRI-Only) Subject: sub-95, MRI Path: C:\Users\ishsh\OneDrive\Desktop\QML_Project\data\processed\mri\derivatives\cleaned_skullstrip\sub-95\anat\sub-95_T1w_skullstripped.nii.gz, ROI Data Shape: (1, 116)
(Reference MRI-Only) Subject: sub-96, MRI Path: C:\Users\ishsh\OneDrive\Desktop\QML_Project\data\processed\mri\derivatives\cleaned_skullstrip\sub-96\ana

c:\Users\ishsh\OneDrive\Desktop\QML_Project\venv4qml\Lib\site-packages\nilearn\maskers\nifti_labels_masker.py:888: UserWarning: After resampling the label image to the data image, the following labels were removed: {5101}. Label image only contains 116 labels (including background).
  warnings.warn(


(Reference MRI-Only) Subject: sub-97, MRI Path: C:\Users\ishsh\OneDrive\Desktop\QML_Project\data\processed\mri\derivatives\cleaned_skullstrip\sub-97\anat\sub-97_T1w_skullstripped.nii.gz, ROI Data Shape: (1, 115)
(Reference MRI-Only) Subject: sub-98, MRI Path: C:\Users\ishsh\OneDrive\Desktop\QML_Project\data\processed\mri\derivatives\cleaned_skullstrip\sub-98\anat\sub-98_T1w_skullstripped.nii.gz, ROI Data Shape: (1, 116)
(Reference MRI-Only) Subject: sub-99, MRI Path: C:\Users\ishsh\OneDrive\Desktop\QML_Project\data\processed\mri\derivatives\cleaned_skullstrip\sub-99\anat\sub-99_T1w_skullstripped.nii.gz, ROI Data Shape: (1, 116)
Surviving MRI-Only labels (after reference): range(0, 116)


c:\Users\ishsh\OneDrive\Desktop\QML_Project\venv4qml\Lib\site-packages\nilearn\maskers\nifti_labels_masker.py:888: UserWarning: After resampling the label image to the data image, the following labels were removed: {5102}. Label image only contains 116 labels (including background).
  warnings.warn(
c:\Users\ishsh\OneDrive\Desktop\QML_Project\venv4qml\Lib\site-packages\nilearn\maskers\nifti_labels_masker.py:888: UserWarning: After resampling the label image to the data image, the following labels were removed: {5101, 5102}. Label image only contains 115 labels (including background).
  warnings.warn(
c:\Users\ishsh\OneDrive\Desktop\QML_Project\venv4qml\Lib\site-packages\nilearn\maskers\nifti_labels_masker.py:888: UserWarning: After resampling the label image to the data image, the following labels were removed: {5101, 5102}. Label image only contains 115 labels (including background).
  warnings.warn(
c:\Users\ishsh\OneDrive\Desktop\QML_Project\venv4qml\Lib\site-packages\nilearn\masker

MRI-PET (Anat) Data Path: C:\Users\ishsh\OneDrive\Desktop\QML_Project\data\processed\mri_pet\derivatives\cleaned_skullstrip
PET (Processed) Data Path: C:\Users\ishsh\OneDrive\Desktop\QML_Project\data\processed\mri_pet\derivatives\preprocessed_pet


# Process MRI and PET (from MRI-PET) data

In [6]:

mri_pet_data = {}
surviving_mri_pet_labels = None
reference_mri_pet_path = None
max_rois_mri_pet = 0

for subject_folder in os.listdir(mri_pet_anat_data_path):
    if subject_folder.startswith("sub-control") or subject_folder.startswith("sub-patient"):
        mri_path = os.path.join(mri_pet_anat_data_path, subject_folder, "anat", f"{subject_folder}_T1w_skullstripped.nii.gz")
        try:
            img = nib.load(mri_path)
            masker = input_data.NiftiLabelsMasker(labels_img=atlas_filename, standardize=False)
            roi_data = masker.fit_transform(img)
            num_rois = roi_data.shape[1]

            if num_rois > max_rois_mri_pet:
                max_rois_mri_pet = num_rois
                reference_mri_pet_path = mri_path
            print(f"(Reference MRI-PET MRI) Subject: {subject_folder}, MRI Path: {mri_path}, ROI Data Shape: {roi_data.shape}") # DEBUG
        except FileNotFoundError:
            logging.warning(f"MRI file not found for subject: {subject_folder} in {mri_path}")
        except Exception as e:
            logging.error(f"Error loading or processing MRI image {mri_path}: {e}")

if reference_mri_pet_path:
    _, surviving_mri_pet_labels = extract_roi_features(reference_mri_pet_path, atlas_filename)
    print(f"Surviving MRI-PET labels (after reference): {surviving_mri_pet_labels}") # DEBUG
else:
    logging.error("No reference MRI found for MRI-PET data. Skipping MRI-PET processing.")

if surviving_mri_pet_labels is not None:
    for subject_folder in os.listdir(mri_pet_anat_data_path):
        if subject_folder.startswith("sub-control") or subject_folder.startswith("sub-patient"):
            subject_id = subject_folder
            mri_path = os.path.join(mri_pet_anat_data_path, subject_folder, "anat", f"{subject_folder}_T1w_skullstripped.nii.gz")
            print(f"(Processing MRI-PET MRI) Subject: {subject_folder}, MRI Path: {mri_path}") # DEBUG
            mri_features, _ = extract_roi_features(mri_path, atlas_filename, surviving_mri_pet_labels)

            pet_folder = os.path.join(pet_processed_data_path, subject_folder, "pet")
            pet_path = None
            try:
                for filename in os.listdir(pet_folder):
                    if filename.endswith("_space-MNI152NLin2009aPET.nii.gz"):
                        pet_path = os.path.join(pet_folder, filename)
                        break
                if pet_path:
                    print(f"(Processing PET) Subject: {subject_folder}, PET Path: {pet_path}") # DEBUG
                    pet_features, _ = extract_roi_features(pet_path, atlas_filename, surviving_mri_pet_labels)
                else:
                    pet_features = None
                    logging.warning(f"PET image not found for subject: {subject_folder} in {pet_folder}")
            except FileNotFoundError:
                pet_features = None
                logging.warning(f"PET folder not found for subject: {subject_folder} at {pet_folder}")

            label = 0 if "control" in subject_folder else 1
            mri_pet_data[subject_id] = {
                **({f'M{i+1}': val for i, val in enumerate(mri_features.flatten())} if mri_features is not None else {}),
                **({f'P{i+1}': val for i, val in enumerate(pet_features.flatten())} if pet_features is not None else {}),
                'label': label
            }


c:\Users\ishsh\OneDrive\Desktop\QML_Project\venv4qml\Lib\site-packages\nilearn\maskers\nifti_labels_masker.py:888: UserWarning: After resampling the label image to the data image, the following labels were removed: {6401, 6402, 4101, 4102, 2311, 8201, 8202, 4111, 4112, 2321, 2322, 8211, 8212, 5401, 5402, 2331, 9001, 9002, 2611, 2612, 2101, 9011, 9012, 6201, 6202, 9021, 9022, 2111, 2112, 6211, 6212, 9031, 9032, 6221, 6222, 5201, 5202, 9041, 9042, 7001, 9051, 9052, 7011, 9061, 9062, 4201, 4202, 8301, 8302, 7021, 9071, 9072, 6001, 9081, 9082, 5001, 5002, 9100, 2701, 2702, 5011, 5012, 9110, 2201, 6301, 6302, 5021, 5022, 9120, 2211, 2212, 8101, 9130, 8111, 9140, 5301, 5302, 4021, 4022, 3001, 8121, 7101, 7102, 9150, 2501, 2502, 9160, 2001, 9170, 6101, 6102, 5101, 5102, 2301}. Label image only contains 23 labels (including background).
  warnings.warn(


(Reference MRI-PET MRI) Subject: sub-control01, MRI Path: C:\Users\ishsh\OneDrive\Desktop\QML_Project\data\processed\mri_pet\derivatives\cleaned_skullstrip\sub-control01\anat\sub-control01_T1w_skullstripped.nii.gz, ROI Data Shape: (1, 22)


c:\Users\ishsh\OneDrive\Desktop\QML_Project\venv4qml\Lib\site-packages\nilearn\maskers\nifti_labels_masker.py:888: UserWarning: After resampling the label image to the data image, the following labels were removed: {6401, 6402, 4101, 4102, 2311, 8201, 8202, 4111, 4112, 2321, 2322, 8211, 8212, 5401, 5402, 2331, 9001, 9002, 2611, 2612, 2101, 9011, 9012, 6201, 6202, 9021, 9022, 2111, 2112, 6211, 6212, 9031, 9032, 6221, 6222, 5201, 5202, 9041, 9042, 7001, 9051, 9052, 7011, 9061, 9062, 4201, 4202, 8301, 8302, 7021, 9071, 9072, 6001, 9081, 9082, 5001, 5002, 9100, 2701, 2702, 5011, 5012, 9110, 2201, 6301, 6302, 5021, 5022, 9120, 2211, 2212, 8101, 9130, 8111, 9140, 5301, 5302, 4021, 4022, 3001, 8121, 7101, 7102, 9150, 2501, 2502, 9160, 2001, 9170, 6101, 6102, 5101, 5102, 2301}. Label image only contains 23 labels (including background).
  warnings.warn(


(Reference MRI-PET MRI) Subject: sub-control02, MRI Path: C:\Users\ishsh\OneDrive\Desktop\QML_Project\data\processed\mri_pet\derivatives\cleaned_skullstrip\sub-control02\anat\sub-control02_T1w_skullstripped.nii.gz, ROI Data Shape: (1, 22)


c:\Users\ishsh\OneDrive\Desktop\QML_Project\venv4qml\Lib\site-packages\nilearn\maskers\nifti_labels_masker.py:888: UserWarning: After resampling the label image to the data image, the following labels were removed: {6401, 6402, 4101, 4102, 2311, 8201, 8202, 4111, 4112, 2321, 2322, 8211, 8212, 5401, 5402, 2331, 9001, 9002, 2611, 2612, 2101, 9011, 9012, 6201, 6202, 9021, 9022, 2111, 2112, 6211, 6212, 9031, 9032, 6221, 6222, 5201, 5202, 9041, 9042, 7001, 9051, 9052, 7011, 9061, 9062, 4201, 4202, 8301, 8302, 7021, 9071, 9072, 6001, 9081, 9082, 5001, 5002, 9100, 2701, 2702, 5011, 5012, 9110, 2201, 6301, 6302, 5021, 5022, 9120, 2211, 2212, 8101, 9130, 8111, 9140, 5301, 5302, 4021, 4022, 3001, 8121, 7101, 7102, 9150, 2501, 2502, 9160, 2001, 9170, 6101, 6102, 5101, 5102, 2301}. Label image only contains 23 labels (including background).
  warnings.warn(


...

(Processing PET) Subject: sub-patient22, PET Path: C:\Users\ishsh\OneDrive\Desktop\QML_Project\data\processed\mri_pet\derivatives\preprocessed_pet\sub-patient22\pet\sub-patient22_space-MNI152NLin2009aPET.nii.gz
(Processing MRI-PET MRI) Subject: sub-patient23, MRI Path: C:\Users\ishsh\OneDrive\Desktop\QML_Project\data\processed\mri_pet\derivatives\cleaned_skullstrip\sub-patient23\anat\sub-patient23_T1w_skullstripped.nii.gz


c:\Users\ishsh\OneDrive\Desktop\QML_Project\venv4qml\Lib\site-packages\nilearn\maskers\nifti_labels_masker.py:888: UserWarning: After resampling the label image to the data image, the following labels were removed: {6401, 6402, 4101, 4102, 2311, 8201, 8202, 4111, 4112, 2321, 2322, 8211, 8212, 5401, 5402, 2331, 9001, 9002, 2611, 2612, 2101, 9011, 9012, 6201, 6202, 9021, 9022, 2111, 2112, 6211, 6212, 9031, 9032, 6221, 6222, 5201, 5202, 9041, 9042, 7001, 9051, 9052, 7011, 9061, 9062, 4201, 4202, 8301, 8302, 7021, 9071, 9072, 6001, 9081, 9082, 5001, 5002, 9100, 2701, 2702, 5011, 5012, 9110, 2201, 6301, 6302, 5021, 5022, 9120, 2211, 2212, 8101, 9130, 8111, 9140, 5301, 5302, 4021, 4022, 3001, 8121, 7101, 7102, 9150, 2501, 2502, 9160, 2001, 9170, 6101, 6102, 5101, 5102, 2301}. Label image only contains 23 labels (including background).
  warnings.warn(


(Processing PET) Subject: sub-patient23, PET Path: C:\Users\ishsh\OneDrive\Desktop\QML_Project\data\processed\mri_pet\derivatives\preprocessed_pet\sub-patient23\pet\sub-patient23_space-MNI152NLin2009aPET.nii.gz
(Processing MRI-PET MRI) Subject: sub-patient24, MRI Path: C:\Users\ishsh\OneDrive\Desktop\QML_Project\data\processed\mri_pet\derivatives\cleaned_skullstrip\sub-patient24\anat\sub-patient24_T1w_skullstripped.nii.gz


c:\Users\ishsh\OneDrive\Desktop\QML_Project\venv4qml\Lib\site-packages\nilearn\maskers\nifti_labels_masker.py:888: UserWarning: After resampling the label image to the data image, the following labels were removed: {6401, 6402, 4101, 4102, 2311, 8201, 8202, 4111, 4112, 2321, 2322, 8211, 8212, 5401, 5402, 2331, 9001, 9002, 2611, 2612, 2101, 9011, 9012, 6201, 6202, 9021, 9022, 2111, 2112, 6211, 6212, 9031, 9032, 6221, 6222, 5201, 5202, 9041, 9042, 7001, 9051, 9052, 7011, 9061, 9062, 4201, 4202, 8301, 8302, 7021, 9071, 9072, 6001, 9081, 9082, 5001, 5002, 9100, 2701, 2702, 5011, 5012, 9110, 2201, 6301, 6302, 5021, 5022, 9120, 2211, 2212, 8101, 9130, 8111, 9140, 5301, 5302, 4021, 4022, 3001, 8121, 7101, 7102, 9150, 2501, 2502, 9160, 2001, 9170, 6101, 6102, 5101, 5102, 2301}. Label image only contains 23 labels (including background).
  warnings.warn(


(Processing PET) Subject: sub-patient24, PET Path: C:\Users\ishsh\OneDrive\Desktop\QML_Project\data\processed\mri_pet\derivatives\preprocessed_pet\sub-patient24\pet\sub-patient24_space-MNI152NLin2009aPET.nii.gz


# --- Save to CSV files ---

In [7]:

def save_list_of_dicts_to_csv(data, filename):
    df = pd.DataFrame(data)
    df.to_csv(filename, index=False)
    print(f"\nData saved to {filename}")

def save_combined_mri_pet_data_to_csv(data, filename):
    output_data = []
    for subject_id, features in data.items():
        output_data.append({'subject_id': subject_id, **features})
    df = pd.DataFrame(output_data)
    df.to_csv(filename, index=False)
    print(f"\nCombined MRI-PET data saved to {filename}")

# Save MRI-only data
save_list_of_dicts_to_csv(mri_only_data, 'mri_only_data_wide.csv')

# Save combined MRI-PET data
save_combined_mri_pet_data_to_csv(mri_pet_data, 'mri_pet_combined_wide.csv')

print("\nProcessing complete. Data saved to wide format CSV files.")

import pandas as pd

# Load and print the head of the new CSV files
try:
    df_mri_only = pd.read_csv('mri_only_data_wide.csv')
    print("\nHead of MRI-Only Data (Wide Format):")
    print(df_mri_only.head())
except FileNotFoundError:
    print("\nError: mri_only_data_wide.csv not found.")

try:
    df_mri_pet = pd.read_csv('mri_pet_combined_wide.csv')
    print("\nHead of Combined MRI-PET Data (Wide Format):")
    print(df_mri_pet.head())
except FileNotFoundError:
    print("\nError: mri_pet_combined_wide.csv not found.")


Data saved to mri_only_data_wide.csv

Combined MRI-PET data saved to mri_pet_combined_wide.csv

Processing complete. Data saved to wide format CSV files.

Head of MRI-Only Data (Wide Format):
  subject_id           M1   M2  M3  M4   M5   M6   M7   M8           M9  ...  \
0      sub-1   477.709106  0.0 NaN NaN  0.0  0.0  0.0  0.0   384.788971  ...   
1     sub-10  3398.151855  0.0 NaN NaN  0.0  0.0  0.0  0.0  2971.952148  ...   
2    sub-100  1006.615845  0.0 NaN NaN  0.0  0.0  0.0  0.0   725.388794  ...   
3    sub-101  1405.849976  0.0 NaN NaN  0.0  0.0  0.0  0.0   775.935669  ...   
4    sub-102   203.207291  0.0 NaN NaN  0.0  0.0  0.0  0.0   105.300316  ...   

   M920      M921  M922  M923  M924  M925  M926  M927  M928  label  
0   0.0  2.157357   0.0   NaN   NaN   0.0   0.0   0.0   0.0      1  
1   0.0       NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN      1  
2   NaN       NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN      1  
3   NaN       NaN   NaN   NaN   NaN   NaN   NaN   

# Cleaning data

In [8]:
import pandas as pd
import os

# Define the directory to save the cleaned files
output_directory = r"C:\Users\ishsh\OneDrive\Desktop\QML_Project\results"

# Create the output directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)
    print(f"Created output directory: {output_directory}")

# Load the MRI-only data (assuming it's in the same directory as the script)
try:
    df_mri_only = pd.read_csv('mri_only_data_wide.csv')
    print("\n--- MRI-Only Data ---")
    total_rows_mri_only = len(df_mri_only)
    columns_to_drop_mri_only = []
    for col in df_mri_only.columns:
        nan_percentage = df_mri_only[col].isnull().sum() / total_rows_mri_only * 100
        if nan_percentage > 50:
            columns_to_drop_mri_only.append(col)

    print(f"Number of columns with > 50% NaN values: {len(columns_to_drop_mri_only)}")
    if columns_to_drop_mri_only:
        print("Columns with > 50% NaN values:", columns_to_drop_mri_only)
        df_mri_only_cleaned = df_mri_only.drop(columns=columns_to_drop_mri_only)
        print("Shape of MRI-Only data after dropping columns:", df_mri_only_cleaned.shape)
    else:
        print("No columns found with > 50% NaN values.")
        df_mri_only_cleaned = df_mri_only.copy()  # Create a copy if no columns were dropped

    # Save the cleaned MRI-only data to the specified output directory
    if df_mri_only_cleaned is not None:
        filepath_mri_only_cleaned = os.path.join(output_directory, 'mri_only_cleaned.csv')
        df_mri_only_cleaned.to_csv(filepath_mri_only_cleaned, index=False)
        print(f"Cleaned MRI-Only data saved to: {filepath_mri_only_cleaned}")
    else:
        print("\ndf_mri_only_cleaned not available, so not saved.")

except FileNotFoundError:
    print("\nError: mri_only_data_wide.csv not found in the current directory.")
    df_mri_only_cleaned = None

# Load the combined MRI-PET data (assuming it's in the same directory as the script)
try:
    df_mri_pet = pd.read_csv('mri_pet_combined_wide.csv')
    print("\n--- Combined MRI-PET Data ---")
    total_rows_mri_pet = len(df_mri_pet)
    columns_to_drop_mri_pet = []
    for col in df_mri_pet.columns:
        nan_percentage = df_mri_pet[col].isnull().sum() / total_rows_mri_pet * 100
        if nan_percentage > 50:
            columns_to_drop_mri_pet.append(col)

    print(f"Number of columns with > 50% NaN values: {len(columns_to_drop_mri_pet)}")
    if columns_to_drop_mri_pet:
        print("Columns with > 50% NaN values:", columns_to_drop_mri_pet)
        df_mri_pet_cleaned = df_mri_pet.drop(columns=columns_to_drop_mri_pet)
        print("Shape of Combined MRI-PET data after dropping columns:", df_mri_pet_cleaned.shape)
    else:
        print("No columns found with > 50% NaN values.")
        df_mri_pet_cleaned = df_mri_pet.copy()  # Create a copy if no columns were dropped

    # Save the cleaned combined MRI-PET data to the specified output directory
    if df_mri_pet_cleaned is not None:
        filepath_mri_pet_cleaned = os.path.join(output_directory, 'mri_pet_combined_cleaned.csv')
        df_mri_pet_cleaned.to_csv(filepath_mri_pet_cleaned, index=False)
        print(f"Cleaned Combined MRI-PET data saved to: {filepath_mri_pet_cleaned}")
    else:
        print("\ndf_mri_pet_cleaned not available, so not saved.")

except FileNotFoundError:
    print("\nError: mri_pet_combined_wide.csv not found in the current directory.")
    df_mri_pet_cleaned = None

print("\nCleaning process complete. Cleaned files saved to the specified directory.")


--- MRI-Only Data ---
Number of columns with > 50% NaN values: 232
Columns with > 50% NaN values: ['M3', 'M4', 'M11', 'M12', 'M19', 'M20', 'M27', 'M28', 'M35', 'M36', 'M43', 'M44', 'M51', 'M52', 'M59', 'M60', 'M67', 'M68', 'M75', 'M76', 'M83', 'M84', 'M91', 'M92', 'M99', 'M100', 'M107', 'M108', 'M115', 'M116', 'M123', 'M124', 'M131', 'M132', 'M139', 'M140', 'M147', 'M148', 'M155', 'M156', 'M163', 'M164', 'M171', 'M172', 'M179', 'M180', 'M187', 'M188', 'M195', 'M196', 'M203', 'M204', 'M211', 'M212', 'M219', 'M220', 'M227', 'M228', 'M235', 'M236', 'M243', 'M244', 'M251', 'M252', 'M259', 'M260', 'M267', 'M268', 'M275', 'M276', 'M283', 'M284', 'M291', 'M292', 'M299', 'M300', 'M307', 'M308', 'M315', 'M316', 'M323', 'M324', 'M331', 'M332', 'M339', 'M340', 'M347', 'M348', 'M355', 'M356', 'M363', 'M364', 'M371', 'M372', 'M379', 'M380', 'M387', 'M388', 'M395', 'M396', 'M403', 'M404', 'M411', 'M412', 'M419', 'M420', 'M427', 'M428', 'M435', 'M436', 'M443', 'M444', 'M451', 'M452', 'M459', 'M460',

# Check NaN values in the cleaned MRI-only DataFrame

In [9]:
import pandas as pd

# Assuming you have run the previous code block to create df_mri_only_cleaned and df_mri_pet_cleaned

if 'df_mri_only_cleaned' in locals() and df_mri_only_cleaned is not None:
    print("\n--- Cleaned MRI-Only Data ---")
    print(f"Total number of columns: {df_mri_only_cleaned.shape[1]}")
    nan_counts_cleaned_mri_only = df_mri_only_cleaned.isnull().sum()
    columns_with_nan_cleaned_mri_only = nan_counts_cleaned_mri_only[nan_counts_cleaned_mri_only > 0]
    num_cols_with_nan_cleaned_mri_only = len(columns_with_nan_cleaned_mri_only)
    print(f"Number of columns with NaN values: {num_cols_with_nan_cleaned_mri_only}")
    if num_cols_with_nan_cleaned_mri_only > 0:
        print("\nNaN values per column:")
        print(nan_counts_cleaned_mri_only[nan_counts_cleaned_mri_only > 0])
    else:
        print("\nNo columns with NaN values found.")
else:
    print("\nError: df_mri_only_cleaned not found. Make sure the previous step ran successfully.")

# Check NaN values in the cleaned combined MRI-PET DataFrame
if 'df_mri_pet_cleaned' in locals() and df_mri_pet_cleaned is not None:
    print("\n--- Cleaned Combined MRI-PET Data ---")
    print(f"Total number of columns: {df_mri_pet_cleaned.shape[1]}")
    nan_counts_cleaned_mri_pet = df_mri_pet_cleaned.isnull().sum()
    columns_with_nan_cleaned_mri_pet = nan_counts_cleaned_mri_pet[nan_counts_cleaned_mri_pet > 0]
    num_cols_with_nan_cleaned_mri_pet = len(columns_with_nan_cleaned_mri_pet)
    print(f"Number of columns with NaN values: {num_cols_with_nan_cleaned_mri_pet}")
    if num_cols_with_nan_cleaned_mri_pet > 0:
        print("\nNaN values per column:")
        print(nan_counts_cleaned_mri_pet[nan_counts_cleaned_mri_pet > 0])
    else:
        print("\nNo columns with NaN values found.")
else:
    print("\nError: df_mri_pet_cleaned not found. Make sure the previous step ran successfully.")


--- Cleaned MRI-Only Data ---
Total number of columns: 698
Number of columns with NaN values: 102

NaN values per column:
M793      1
M794      1
M797      1
M798      1
M799      1
       ... 
M922    133
M925    133
M926    133
M927    133
M928    133
Length: 102, dtype: int64

--- Cleaned Combined MRI-PET Data ---
Total number of columns: 266
Number of columns with NaN values: 0

No columns with NaN values found.


# viewing final data aspects

In [2]:
import pandas as pd
import os

# Define the directory to save the cleaned files
output_directory = r"C:\Users\ishsh\OneDrive\Desktop\QML_Project\results"

# Create the output directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)
    print(f"Created output directory: {output_directory}")

# --- Load and Process MRI-Only Data ---
try:
    df_mri_only = pd.read_csv('mri_only_data_wide.csv')
    print("\n--- MRI-Only Data ---")
    print(f"Shape of original MRI-Only data: {df_mri_only.shape}")

    columns_to_drop_mri_only = []
    for col in df_mri_only.columns:
        nan_count = df_mri_only[col].isnull().sum()
        if nan_count > 40:
            columns_to_drop_mri_only.append(col)

    print(f"Number of columns with > 40 NaN values: {len(columns_to_drop_mri_only)}")
    if columns_to_drop_mri_only:
        print("Columns with > 40 NaN values:", columns_to_drop_mri_only)
        df_mri_only_cleaned = df_mri_only.drop(columns=columns_to_drop_mri_only)
        print(f"Shape of cleaned MRI-Only data (after dropping > 40 NaNs): {df_mri_only_cleaned.shape}")
    else:
        print("No columns found with > 40 NaN values.")
        df_mri_only_cleaned = df_mri_only.copy()
        print(f"Shape of cleaned MRI-Only data (no change): {df_mri_only_cleaned.shape}")

    # Save the cleaned MRI-only data
    if df_mri_only_cleaned is not None:
        filepath_mri_only_cleaned = os.path.join(output_directory, 'mri_only_cleaned.csv')
        df_mri_only_cleaned.to_csv(filepath_mri_only_cleaned, index=False)
        print(f"Cleaned MRI-Only data saved to: {filepath_mri_only_cleaned}")
    else:
        print("\ndf_mri_only_cleaned not available, so not saved.")

except FileNotFoundError:
    print("\nError: mri_only_data_wide.csv not found in the current directory.")
    df_mri_only_cleaned = None

# --- Load and Process Combined MRI-PET Data ---
try:
    df_mri_pet = pd.read_csv('mri_pet_combined_wide.csv')
    print("\n--- Combined MRI-PET Data ---")
    print(f"Shape of original Combined MRI-PET data: {df_mri_pet.shape}")

    columns_to_drop_mri_pet = []
    for col in df_mri_pet.columns:
        nan_count = df_mri_pet[col].isnull().sum()
        if nan_count > 40:
            columns_to_drop_mri_pet.append(col)

    print(f"Number of columns with > 40 NaN values: {len(columns_to_drop_mri_pet)}")
    if columns_to_drop_mri_pet:
        print("Columns with > 40 NaN values:", columns_to_drop_mri_pet)
        df_mri_pet_cleaned = df_mri_pet.drop(columns=columns_to_drop_mri_pet)
        print(f"Shape of cleaned Combined MRI-PET data (after dropping > 40 NaNs): {df_mri_pet_cleaned.shape}")
    else:
        print("No columns found with > 40 NaN values.")
        df_mri_pet_cleaned = df_mri_pet.copy()
        print(f"Shape of cleaned Combined MRI-PET data (no change): {df_mri_pet_cleaned.shape}")

    # Save the cleaned combined MRI-PET data
    if df_mri_pet_cleaned is not None:
        filepath_mri_pet_cleaned = os.path.join(output_directory, 'mri_pet_combined_cleaned.csv')
        df_mri_pet_cleaned.to_csv(filepath_mri_pet_cleaned, index=False)
        print(f"Cleaned Combined MRI-PET data saved to: {filepath_mri_pet_cleaned}")
    else:
        print("\ndf_mri_pet_cleaned not available, so not saved.")

except FileNotFoundError:
    print("\nError: mri_pet_combined_wide.csv not found in the current directory.")
    df_mri_pet_cleaned = None

print("\nProcessing complete. Data shapes and cleaned files handled.")


--- MRI-Only Data ---
Shape of original MRI-Only data: (542, 930)
Number of columns with > 40 NaN values: 250
Columns with > 40 NaN values: ['M3', 'M4', 'M11', 'M12', 'M19', 'M20', 'M27', 'M28', 'M35', 'M36', 'M43', 'M44', 'M51', 'M52', 'M59', 'M60', 'M67', 'M68', 'M75', 'M76', 'M83', 'M84', 'M91', 'M92', 'M99', 'M100', 'M107', 'M108', 'M115', 'M116', 'M123', 'M124', 'M131', 'M132', 'M139', 'M140', 'M147', 'M148', 'M155', 'M156', 'M163', 'M164', 'M171', 'M172', 'M179', 'M180', 'M187', 'M188', 'M195', 'M196', 'M203', 'M204', 'M211', 'M212', 'M219', 'M220', 'M227', 'M228', 'M235', 'M236', 'M243', 'M244', 'M251', 'M252', 'M259', 'M260', 'M267', 'M268', 'M275', 'M276', 'M283', 'M284', 'M291', 'M292', 'M299', 'M300', 'M307', 'M308', 'M315', 'M316', 'M323', 'M324', 'M331', 'M332', 'M339', 'M340', 'M347', 'M348', 'M355', 'M356', 'M363', 'M364', 'M371', 'M372', 'M379', 'M380', 'M387', 'M388', 'M395', 'M396', 'M403', 'M404', 'M411', 'M412', 'M419', 'M420', 'M427', 'M428', 'M435', 'M436', 'M443

# viewing dataset shape

In [3]:
import pandas as pd

# --- Load and Get Shape of Original MRI-Only Data ---
try:
    df_mri_only = pd.read_csv('mri_only_data_wide.csv')
    print(f"Shape of original MRI-Only data: {df_mri_only.shape}")
except FileNotFoundError:
    print("Error: mri_only_data_wide.csv not found.")
    df_mri_only = None

# --- Load and Get Shape of Original Combined MRI-PET Data ---
try:
    df_mri_pet = pd.read_csv('mri_pet_combined_wide.csv')
    print(f"Shape of original Combined MRI-PET data: {df_mri_pet.shape}")
except FileNotFoundError:
    print("Error: mri_pet_combined_wide.csv not found.")
    df_mri_pet = None

# --- Load Cleaned MRI-Only Data and Get Shape ---
try:
    df_mri_only_cleaned = pd.read_csv(r'C:/Users/ishsh/OneDrive/Desktop/QML_Project/results/mri_only_cleaned.csv')
    print(f"Shape of cleaned MRI-Only data: {df_mri_only_cleaned.shape}")
except FileNotFoundError:
    print("Error: mri_only_cleaned.csv not found.")
    df_mri_only_cleaned = None

# --- Load Cleaned Combined MRI-PET Data and Get Shape ---
try:
    df_mri_pet_cleaned = pd.read_csv(r'C:/Users/ishsh/OneDrive/Desktop/QML_Project/results/mri_pet_combined_cleaned.csv')
    print(f"Shape of cleaned Combined MRI-PET data: {df_mri_pet_cleaned.shape}")
except FileNotFoundError:
    print("Error: mri_pet_combined_cleaned.csv not found.")
    df_mri_pet_cleaned = None

Shape of original MRI-Only data: (542, 930)
Shape of original Combined MRI-PET data: (42, 354)
Shape of cleaned MRI-Only data: (542, 680)
Shape of cleaned Combined MRI-PET data: (42, 266)


In [6]:
import pandas as pd

def analyze_nan_distribution(df, df_name):
    """Analyzes and prints the distribution of NaN values per column in a DataFrame."""
    if df is not None:
        print(f"\n--- NaN Distribution in {df_name} ---")
        nan_counts = df.isnull().sum()
        columns_with_nan = nan_counts[nan_counts > 0].sort_values()

        if columns_with_nan.empty:
            print("No columns with NaN values found.")
            return

        nan_ranges = {}
        for i in range(1, int(columns_with_nan.max() // 10) + 2):
            lower_bound = (i - 1) * 10 + 1
            upper_bound = i * 10
            if i == int(columns_with_nan.max() // 10) + 1 and columns_with_nan.max() % 10 != 0:
                upper_bound = float('inf')
            nan_ranges[f"{lower_bound}-{upper_bound if upper_bound != float('inf') else 'inf'}"] = []

        for col, nan_count in columns_with_nan.items():
            for range_str, col_list in nan_ranges.items():
                lower, upper_str = range_str.split('-')
                upper = float(upper_str)
                if lower <= str(nan_count) and nan_count <= upper:
                    col_list.append(col)
                    break

        for range_str, col_list in nan_ranges.items():
            if col_list:
                print(f"\nColumns with NaN values in the range {range_str} (Count: {len(col_list)}):")
                print(col_list)
    else:
        print(f"\nError: DataFrame {df_name} is not defined.")

# Assuming you have run the previous code block to create df_mri_only_cleaned and df_mri_pet_cleaned

if 'df_mri_only_cleaned' in locals():
    analyze_nan_distribution(df_mri_only_cleaned, 'Cleaned MRI-Only Data')

if 'df_mri_pet_cleaned' in locals():
    analyze_nan_distribution(df_mri_pet_cleaned, 'Cleaned Combined MRI-PET Data')


--- NaN Distribution in Cleaned MRI-Only Data ---

Columns with NaN values in the range 1-10 (Count: 60):
['M793', 'M794', 'M797', 'M798', 'M799', 'M800', 'M816', 'M815', 'M814', 'M813', 'M810', 'M809', 'M808', 'M807', 'M806', 'M805', 'M802', 'M801', 'M832', 'M831', 'M830', 'M829', 'M826', 'M825', 'M817', 'M823', 'M822', 'M821', 'M818', 'M824', 'M833', 'M834', 'M837', 'M838', 'M839', 'M840', 'M847', 'M846', 'M848', 'M842', 'M841', 'M845', 'M849', 'M850', 'M853', 'M854', 'M855', 'M856', 'M864', 'M863', 'M862', 'M861', 'M857', 'M858', 'M865', 'M866', 'M869', 'M870', 'M871', 'M872']

Columns with NaN values in the range 11-20 (Count: 6):
['M880', 'M879', 'M878', 'M874', 'M873', 'M877']

Columns with NaN values in the range 21-30 (Count: 12):
['M881', 'M882', 'M885', 'M886', 'M887', 'M888', 'M896', 'M895', 'M894', 'M890', 'M889', 'M893']

Columns with NaN values in the range 31-inf (Count: 6):
['M903', 'M897', 'M898', 'M901', 'M902', 'M904']

--- NaN Distribution in Cleaned Combined MRI-P